In [1]:
%load_ext blackcellmagic

In [2]:
import os, pdb
import pandas as pd
import seaborn as sns
import numpy as np
from typing import List, Tuple
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

# Parameters

In [3]:
PROJECT_BASE_DIR = "/home/rohail/projects/imdb_ratings/"
data_dir = "data/raw/"
plot_write_dir = "reports/figures"
idx_columns = ["imdb_title_id", "title", "original_title"]

# Exploratory analysis

## Movies

In [4]:
df_movies = pd.read_csv(os.path.join(PROJECT_BASE_DIR, data_dir, "movies.csv"))
df_movies.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,NaN,24.0,3.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,NaN,28.0,14.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,NaN,12.0,5.0


### Country

In [5]:
sum(df_movies.country.isna())

39

In [6]:
df_movies.loc[:, "primary_country"] = df_movies.country.str.split(",").apply(
    lambda s: s[0] if isinstance(s, list) else s
)
df_movies.loc[:, "primary_country"] = df_movies.primary_country.fillna("not_available")
df_movies.primary_country.value_counts().head()

USA       29712
India      5658
UK         5573
France     5043
Italy      3548
Name: primary_country, dtype: int64

In [7]:
def collapse_categorical_variable(df: pd.DataFrame, column: str, percentage_cutoff: float = 80.0) -> pd.DataFrame:
    
    assert sum(df[column].isna()) == 0
    proportion = (
    df_movies[column].value_counts().sort_values(ascending=False).reset_index()
    )
    proportion.loc[:, "cumpercentage"] = (
        proportion[column].cumsum()
        / proportion[column].sum()
        * 100
    )
    values_to_consider = proportion.loc[proportion["cumpercentage"] < percentage_cutoff,
                                                     "index"].to_list()
    df.loc[~df[column].isin(values_to_consider), column] = "other"
    print(df[column].value_counts())
    return df
    

In [8]:
df_movies = collapse_categorical_variable(df_movies, "primary_country")

USA            29712
other          16962
India           5658
UK              5573
France          5043
Italy           3548
Japan           3043
Canada          2385
Germany         1969
Spain           1687
Hong Kong       1490
Turkey          1331
South Korea     1119
Australia        912
Russia           841
Name: primary_country, dtype: int64


### Language

In [9]:
sum(df_movies.language.isna())

755

In [10]:
df_movies.loc[:, "primary_language"] = df_movies.language.str.split(",").apply(
    lambda s: s[0] if isinstance(s, list) else s
)
df_movies.loc[:, "primary_language"] = df_movies.primary_language.fillna("not_available")
df_movies.primary_language.value_counts().head()

English     41036
French       5184
Spanish      3197
Italian      3071
Japanese     2941
Name: primary_language, dtype: int64

In [11]:
df_movies = collapse_categorical_variable(df_movies, "primary_language")

English     41036
other       17129
French       5184
Spanish      3197
Italian      3071
Japanese     2941
Hindi        2429
German       2291
Russian      1529
Turkish      1330
Mandarin     1136
Name: primary_language, dtype: int64


### Genre

In [12]:
sum(df_movies.genre.isna())

0

In [13]:
df_movies.loc[:, "primary_genre"] = df_movies.genre.str.split(",").apply(
    lambda s: s[0] if isinstance(s, list) else s
)
len(df_movies.primary_genre.value_counts())

23

### Global/local 
For foreign movies, check if it has been released in US (global vs local) or worldwide...


In [14]:
def global_movie(row):
    def is_non_us_grossing_movie(row):
        return ((pd.notnull(row["usa_gross_income"])) & (row["primary_country"] != "USA"))

    def is_worldwide_grossing_movie(row):
        return True if isinstance(row["worlwide_gross_income"], str) else False

    def is_released_in_multiple_locations(row):
        return "," in row["country"] if isinstance(row["country"], str) else False

    flag = (
        (is_non_us_grossing_movie(row))
        | (is_worldwide_grossing_movie(row))
        | (is_released_in_multiple_locations(row))
    )
    return flag


df_movies.loc[:, "global"] = df_movies.apply(global_movie, axis= "columns")
df_movies["global"].value_counts()

False    44489
True     36784
Name: global, dtype: int64

### User/critics reviews

In [15]:
missing_user_critic_review_mask = (
    df_movies[["reviews_from_users", "reviews_from_critics"]].isna().any("columns")
)
(missing_user_critic_review_mask.sum())/len(df_movies) 

0.18173317091777097

In [16]:
def missing_user_critic_review(df):
    return (df.isna().any("columns").sum())/len(df)

df_na_check = df_movies.groupby(["primary_country", "primary_language", "primary_genre"])[
    "reviews_from_users", "reviews_from_critics"
].apply(missing_user_critic_review).reset_index()
df_na_check.rename(columns = {0: "prop_missing"}, inplace = True)
df_na_check = df_na_check.loc[df_na_check["prop_missing"] > 0].copy()
df_na_check.sort_values("prop_missing", ascending = False).head()

,primary_country,primary_language,primary_genre,prop_missing
452,Russia,Russian,Sport,1.0
44,Canada,English,Music,1.0
800,other,Japanese,Music,1.0
797,other,Japanese,Comedy,1.0
786,other,Hindi,Drama,1.0


In [17]:
len(df_na_check.loc[df_na_check.prop_missing == 1])/len(df_na_check)

0.09218436873747494

In [18]:
df_na_check.primary_country.value_counts()

other          108
India           40
Italy           38
USA             36
France          35
Spain           33
Germany         33
Canada          32
UK              25
Turkey          24
Japan           23
Russia          21
Hong Kong       18
South Korea     17
Australia       16
Name: primary_country, dtype: int64

In [19]:
df_na_check.primary_genre.value_counts()

Drama        63
Comedy       51
Action       39
Crime        36
Animation    35
Adventure    33
Biography    30
Thriller     26
Horror       26
Romance      24
Mystery      19
Musical      19
Family       18
Fantasy      16
Sci-Fi       14
War          12
History      11
Music        11
Western      10
Sport         6
Name: primary_genre, dtype: int64

In [20]:
df_na_check.primary_language.value_counts()

English     138
other       107
French       41
Spanish      39
Russian      33
German       33
Italian      26
Turkish      22
Japanese     22
Mandarin     20
Hindi        18
Name: primary_language, dtype: int64

> No specific pattern of missingness - Can remove these rows

In [21]:
df_movies.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics', 'primary_country',
       'primary_language', 'primary_genre', 'global'],
      dtype='object')

In [22]:
# before removing these rows... do we lose any good samples for positive examples?

ratings_with_missing_review = df_movies.loc[missing_user_critic_review_mask, "avg_vote"]
(ratings_with_missing_review >= 7.5).value_counts()

False    13670
True      1100
Name: avg_vote, dtype: int64

In [23]:
sum(df_movies.loc[~missing_user_critic_review_mask, "avg_vote"] >= 7.5)/len(df_movies.loc[~missing_user_critic_review_mask, "avg_vote"])

0.06910966422567404

> Class imabalance slightly increased

In [24]:
df_na_check.prop_missing.describe()

count    499.000000
mean       0.321574
std        0.281082
min        0.004878
25%        0.104356
50%        0.230769
75%        0.500000
max        1.000000
Name: prop_missing, dtype: float64

In [25]:
df_na_check.loc[
    (df_na_check.primary_country != "other") & (df_na_check.primary_language != "other")
].sort_values("prop_missing", ascending=False)

,primary_country,primary_language,primary_genre,prop_missing
417,Japan,Russian,Animation,1.000000
590,Turkey,Turkish,Sport,1.000000
396,Japan,Italian,Sci-Fi,1.000000
392,Japan,English,War,1.000000
390,Japan,English,Musical,1.000000
584,Turkey,Turkish,History,1.000000
98,France,English,Romance,1.000000
582,Turkey,Turkish,Family,1.000000
370,Italy,Turkish,Action,1.000000
571,Turkey,French,Drama,1.000000


In [26]:
len(df_na_check)

499

### Votes

In [27]:
df_movies.loc[:, "vote_popularity"] = pd.qcut(
    df_movies.votes, q=4
).cat.rename_categories(["low_q25", "low_mid_q50", "mid_high_q75", "high_q99"]).astype(str)

In [28]:
df_movies.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language', 'director', 'writer',
       'production_company', 'actors', 'description', 'avg_vote', 'votes',
       'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics', 'primary_country',
       'primary_language', 'primary_genre', 'global', 'vote_popularity'],
      dtype='object')

__TODO:__    
Movies:   
* Normalize budget column to the same amount, but this needs to account for past currencies (pre euro...)
* interact vote, budget, country, genre between each other 
* 'budget', 'usa_gross_income', 'worlwide_gross_income', to be worked with   
* production_company to be accounted for


# Add features back to table

In [ ]:
def enrich_with_features(
    df: pd.DataFrame, idx_columns: List[str], target_columns: List[str], dummy: bool
) -> pd.DataFrame:
    if dummy:
        cols2keep = ["year", "duration"]
    else:
        cols2keep = [
            "year",
            "duration",
            "reviews_from_users",
            "reviews_from_critics",
            "primary_country",
            "primary_language",
            "primary_genre",
            "global",
            "vote_popularity",
        ]
    select = idx_columns + cols2keep + target_columns

    return df.loc[:, select].copy()

target_columns = ["avg_vote"]
df_subset = enrich_with_features(df, idx_columns, target_columns, dummy=False)

In [30]:
df_subset.columns

Index(['imdb_title_id', 'title', 'original_title', 'year', 'duration',
       'reviews_from_users', 'reviews_from_critics', 'primary_country',
       'primary_language', 'primary_genre', 'global', 'vote_popularity',
       'avg_vote'],
      dtype='object')

In [32]:
df_subset.select_dtypes("object").columns

Index(['imdb_title_id', 'title', 'original_title', 'primary_country',
       'primary_language', 'primary_genre', 'vote_popularity'],
      dtype='object')

In [37]:
def prepare_data_for_sklearn(
    df: pd.DataFrame, dummy_encoding: bool, target_variable: str, idx_columns: List[str]
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Apply scaling/encoding to numeric and categorical variables


    Returns:
        A pd.DataFrame, ready to pass to modeling as is.
    """

    # drop key columns and target variable
    X, y = (df.drop(columns=idx_columns + [target_variable]), df[target_variable])

    # determine column types so appropriate transformation is applied
    col2type = {col: typ.name for col, typ in X.dtypes.iteritems()}
    type2cols = defaultdict(list)
    for k, v in col2type.items():
        if v == "object":
            type2cols["string"].append(k)
        else:
            type2cols["numeric"].append(k)

    # categorical variables

    string_cols = type2cols.get("string")
    if dummy_encoding and string_cols:
        cat_features = pd.get_dummies(
            X[string_cols], columns=string_cols, drop_first=True
        )
        X = X.drop(columns=string_cols)  # drop original categorical column
        assert len(X.index) == len(cat_features)
        X = pd.concat([X, cat_features], axis=1)

    # numeric columns

    numeric_cols = type2cols.get("numeric")
    standard_scaler = StandardScaler()
    scaled_features = X.loc[:, numeric_cols].copy()
    scaled_X = standard_scaler.fit_transform(scaled_features.values)
    scaled_df = X.copy()
    scaled_df.loc[:, numeric_cols] = scaled_X

    # add key columns back to scaled_df...
    scaled_df = scaled_df.merge(y, left_index=True, right_index=True).merge(
        df[idx_columns], left_index=True, right_index=True
    )

    unscaled_df = X.merge(y, left_index=True, right_index=True).merge(
        df[idx_columns], left_index=True, right_index=True
    )

    return scaled_df, unscaled_df

In [44]:
scaled_df, unscaled_df = prepare_data_for_sklearn(df_subset,dummy_encoding = True, target_variable = "avg_vote",
                        idx_columns = idx_columns)

# Add further features

Ideas for features:
* Cast and combination of casts is important -> get average rating for actors in cast, same for producers such as tarrantino --> this should probably be by country    
* Capture genre bias on ratings-  comedy is typically rated less - action is typically more    
* Nominations for awards are important (for example parasite), country inflation is a factor --> some countries would tend to rate movies higher
* Focus on movies file first and then consider the actors part -->    
    * ratings has target groups information which I don't need to include in a first version-- has missing values and stuff   
    * actors comes in a second version   
* How polarised is the voting (mean vs median of ratings) -> These movies will probably present more challenges

Different types of movies: 
* English Comedy: will ferrel, big lebowski? 
* Low budget/indie movies:   
* Superhero movies (high budget, high viewership):  
* Global vs local viewership
* Famous producer/actor movies  

## Ratings

In [ ]:
df_ratings = pd.read_csv(os.path.join(PROJECT_BASE_DIR, data_dir, "ratings.csv"))

## Actors

In [ ]:
df_names = pd.read_csv(os.path.join(PROJECT_BASE_DIR, data_dir, "names.csv"))

## Title principals

In [ ]:
df_title_pcpls = pd.read_csv(os.path.join(PROJECT_BASE_DIR, data_dir, "title_principals.csv"))